In [1]:
from Electromagnetics.FDTD import *
import numpy as np
import time

n_dim = 1
r0, r1, dr = [None]*3
S = 0.9
n_t = None
save_every = 50
source_at_right = False

r_c = None
r0_eps, r1_eps = None, None
r0_deps, r1_deps = None, None
a_eps, b_eps = None, None
omega = 2.0*np.pi
s_pml = None
d_pml = None
set_pml = True
is_t_var = True
set_sheet_src = False

L_0 = 2.0*np.pi/omega
if n_dim==1:
    L_0 = 1.0/omega

set_pbc = False and (not set_pml)

if n_dim==3:
    r0 = np.array([0.0, 0.0, 0.0])
    r1 = np.array([2.0, 2.0, 2.0])
    dr = np.array([0.02, 0.02, 0.02])
    S /= np.sqrt(3)
    n_t = 200
    
    r_c = (r0+r1)/2.0
    w_eps = (r1-r0)
    w_eps[1] /= 10.0
    w_eps[2] /= 10.0
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    a_eps, b_eps = 10.0, 1.0
    
    d_pml = np.array([0.2, 0.2, 0.2])
    s_pml = np.array([1.0+1.0j, 1.0+1.0j, 1.0+1.0j])
elif n_dim==2:
    set_wg_pec = True
    a_eps, b_eps = 2.0, 1.0

    w_wg_pec = L_0/0.9/np.sqrt(a_eps)
    r0 = np.array([0.0, 0.0])
    r1 = np.array([15.0*L_0, 2.0*L_0])
    if set_wg_pec:
        r1[1] = w_wg_pec
    dr = np.array([L_0/100, L_0/100])
    S /= np.sqrt(2)
    n_t = 5000

    r_c = (r0+r1)/2.0
    w_eps = (r1-r0)
    w_eps[1] = L_0/1.5/np.sqrt(a_eps)
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    if set_wg_pec:
        w_eps = (r1-r0)
        r0_eps, r1_eps = r0, r1

    d_source = np.array([3.0*L_0, 0])
    w_deps = np.array([r1_eps[0] - 2*d_source[0], 0.5*w_eps[1]])
    r0_deps = r0_eps + d_source
    r1_deps = r0_deps + w_deps

    d_pml = np.array([L_0/2.0, L_0/4.0])
    s_pml = np.array([1.0+6.0j/L_0, 1.0+6.0j/L_0])
    if set_wg_pec:
        d_pml[1] = 0.0
        s_pml[1] = 1.0

elif n_dim==1:
    r0 = np.array([0.0])
    r1 = np.array([40.0*L_0])
    dr = np.array([L_0/100])
    S /= np.sqrt(1)
    n_t = 10000#2**18

    r_c = (r0+r1)/2.0
    a_eps, b_eps = 2.0, 1.0
    w_eps = (r1-r0)/2
    r0_eps = r_c - w_eps/2.0
    r1_eps = r_c + w_eps/2.0
    
    r0_deps = r0_eps
    r1_deps = r1_eps

    d_pml = np.array([0.5*L_0])
    s_pml = np.array([1.0+5.0j/L_0])
    
dt = S*dr[0]
d_v = np.prod(dr)


### Long script

In [16]:

tic = time.clock()

fdtd = FDTDYEE(r0, r1, dr, dt)

print('fdtd.dr:', fdtd.dr)

fdtd.AllocateSideArr_list(["E", "D", "Je", "eps"])
fdtd.AllocateFaceArr_list(["H", "Jm"])
fdtd.SetSpatialPointsSide()
fdtd.SetSpatialPointsFace()

d_x_ = np.zeros(n_dim)
d_x_[0] = (r1[0]-r0[0])
d_x_pml = np.zeros(n_dim)
d_x_pml[0] = d_pml[0]

## J
r_j = r_c - d_x_*0.45 + set_pml*d_x_pml
je_mag = 1.0/dr[0]
if source_at_right:
    r_j = r_c + d_x_*0.45 - set_pml*d_x_pml
f_je = fdtd.GetPointSourceFunc(r_j, mag=je_mag, src_dir='z', em_type='e')
if set_sheet_src:
    assert n_dim>=2
    f_je = fdtd.GetSheetSourceFunc(r_j, mag=lambda r: je_mag*np.cos(r[1]-r_j[1]), src_dir='z', norm_dir='x', em_type='e')
Je_0 = fdtd.UpdateFuncSide_space(fdtd.Je, f_je, getCopy=True)
je_0_max = [np.max(np.abs(j)) for j in Je_0]
print('je_0_max:', je_0_max)

## M
r_m = r_c - d_x_*0.45 + set_pml*d_x_pml
jm_mag = -1.0/dr[0]
if source_at_right:
    r_m = r_c + d_x_*0.45 - set_pml*d_x_pml
    jm_mag *= -1.0
f_jm = fdtd.GetPointSourceFunc(r_j, mag=jm_mag, src_dir='y', em_type='m')
if set_sheet_src:
    assert n_dim>=2
    f_jm = fdtd.GetSheetSourceFunc(r_j, mag=lambda r: jm_mag*np.cos(r[1]-r_j[1]), src_dir='y', norm_dir='x', em_type='m')
Jm_0 = fdtd.UpdateFuncFace_space(fdtd.Jm, f_jm, getCopy=True)
jm_0_max = [np.max(np.abs(m)) for m in Jm_0]
print('jm_0_max:', jm_0_max)


fj_t = lambda t: np.sin(omega*t) 

f_eps = fdtd.FunctionBox(r0_eps, r1_eps, a_eps, b_eps)
eps = fdtd.UpdateFuncSide_space(fdtd.eps, [f_eps]*3)

deps = None
f_deps = None
deps_out = []
if is_t_var:
    fdtd.AllocateSideArr_list(["deps"])
    del_eps = 0.2
    omega_m = 0.2*omega
    k_m = omega_m*np.sqrt(a_eps)
    if n_dim==2:
        k_ = ppwg_disp(omega, d=w_eps[1], eps_r=a_eps, n=1)
        k_m = -0.1*np.real(k_)
        dw = ppwg_dw(k=k_, dk=k_m, d=w_eps[1], eps_r=a_eps, n=[1,2])
        omega_m = np.real(dw)
        #omega_m = -2.0*omega - np.real(dw)
        print('cutoffs', [ppwg_cutoff(r1[1], a_eps, i) for i in range(n_mode)])
    print('del_eps:', del_eps, 'omega_m:', omega_m, 'k_m:', k_m)
    a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r[0])
    if n_dim==1:
        a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r)
    f_deps = fdtd.FunctionBox(r0_deps, r1_deps, a_deps, b=0.0)
    fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t=0.0)


if set_pml:
    fdtd.AllocateSideArr_list(["dF_D", "F_D"])
    fdtd.AllocateFaceArr_list(["dF_H", "F_H"])     

set_eps_out = True
set_j_out = False
set_T_1d = True
set_R_1d = True

r_R_1d = r_j - (r_j-d_pml)/2.0
r_T_1d = r1 - r_R_1d
if source_at_right:
    r_R_1d = r_j + (r1-d_pml-r_j)/2.0
    r_T_1d = r1 - r_R_1d
print('r0:', r0, 'r1:', r1)
print('r_T_1d:', r_T_1d, 'r_R_1d:', r_R_1d)


if n_dim==3:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j,  [{'A':fdtd.Je, 'A_dir':'z', 'O_dir':'x', 'name':'J'}])
elif n_dim<=2:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':None, 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.Je, 'A_dir':'z', 'O_dir':None, 'name':'J'}])
    if n_dim==1 and set_T_1d:
        fdtd.SetViewPlane_Side(r_T_1d, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'T'}])
    if n_dim==1 and set_R_1d:
        fdtd.SetViewPlane_Side(r_R_1d, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'R'}])
        
if is_t_var and set_eps_out:
    if n_dim==3:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':'x', 'name':'Eps'}])
    elif n_dim<=2:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':None, 'name':'Eps'}])


n_saved = 0
        
if not set_pml:
    for i in range(n_t):
        t = float(i*dt)
        ## Je
        fdtd.UpdateSeperableFunc_Time(fdtd.Je, Je_0, fj_t, t)
        ## eps*dE/dt = curl H - Je
        if not is_t_var:
            fdtd.UpdateSide_dAdt_CurlB_C(fdtd.E, fdtd.H, C_list=[fdtd.Je], a=fdtd.eps, b=None, c_list=[-np.ones(3)])
            
            #equivalent to last line
            #curl_H = fdtd.GetCurlFace(fdtd.H)
            #fdtd.Update_adAdt_bB(fdtd.E, curl_H, C_list=[fdtd.Je], a=fdtd.eps, c_list=[-np.ones(3)])

        else:
            fdtd.UpdateSide_dAdt_CurlB_C(fdtd.D, fdtd.H, C_list=[fdtd.Je], a=None, b=None, c_list=[-np.ones(3)])
            fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t)
            eps_p_deps = [fdtd.eps[i]+fdtd.deps[i] for i in range(3)]
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=eps_p_deps)
            

        
        ## PEC
        fdtd.ResetSideWalls(fdtd.E)
        t += dt/2
        ## Jm
        fdtd.UpdateSeperableFunc_Time(fdtd.Jm, Jm_0, fj_t, t)
        ## dH/dt = -curl E - Jm
        fdtd.UpdateFace_dAdt_CurlB_C(fdtd.H, fdtd.E, C_list=[fdtd.Jm], a=None, b=-np.ones(3), c_list=[-np.ones(3)])

        #equivalent to last line
        #curl_E = fdtd.GetCurlSide(fdtd.E)
        #fdtd.Update_adAdt_bB(fdtd.H, curl_E, C_list=[fdtd.Jm], b=-np.ones(3), c_list=[-np.ones(3)])

        if i%save_every==0:
            fdtd.SaveOutputs()
            n_saved += 1
else:
    walls = fdtd.GetWallsAllDic(d_pml, s_pml)
    print('walls:' , walls)
    sx_arr_s, sy_arr_s, sz_arr_s = fdtd.GetUPMLFactor(walls, eps_or_s='s', side_or_face='side')
    sx_arr_f, sy_arr_f, sz_arr_f = fdtd.GetUPMLFactor(walls, eps_or_s='s', side_or_face='face')
            
    c_fd0 = [-np.imag(sy_arr_s[0]), -np.imag(sz_arr_s[1]), -np.imag(sx_arr_s[2])]
    c_fd1 = [np.imag(sx_arr_s[0]), np.imag(sy_arr_s[1]), np.imag(sz_arr_s[2])]
    c_d = [-np.imag(sz_arr_s[0]), -np.imag(sx_arr_s[1]), -np.imag(sy_arr_s[2])]

    c_fh0 = [-np.imag(sy_arr_f[0]), -np.imag(sz_arr_f[1]), -np.imag(sx_arr_f[2])]
    c_fh1 = [np.imag(sx_arr_f[0]), np.imag(sy_arr_f[1]), np.imag(sz_arr_f[2])]
    c_h = [-np.imag(sz_arr_f[0]), -np.imag(sx_arr_f[1]), -np.imag(sy_arr_f[2])]

    #plt.imshow(np.imag(sz_arr_s[0]))
    #plt.show()
    
    for i in range(n_t):
        t = float(i*dt)
        ## Je
        fdtd.UpdateSeperableFunc_Time(fdtd.Je, Je_0, fj_t, t)
        ## curl H
        curl_H = fdtd.GetCurlFace(fdtd.H)
        ## dF_D/dt + s_fd*F_D = curl H - Je
        fdtd.Update_aA_bB(fdtd.dF_D, curl_H, C_list=[fdtd.F_D, fdtd.Je], c_list=[c_fd0, -np.ones(3)])
        ## dD/dt + s_d*D = dF_D/dt + s_fd*F_D
        fdtd.Update_adAdt_bB(fdtd.D, fdtd.dF_D, C_list=[fdtd.D, fdtd.F_D], c_list=[c_d, c_fd1])
        fdtd.Update_adAdt_bB(fdtd.F_D, fdtd.dF_D)
        ## eps*E = D
        if not is_t_var:
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=fdtd.eps)
        else:
            fdtd.UpdateFuncSide_spacetime(fdtd.deps, [f_deps]*3, t)
            eps_p_deps = [fdtd.eps[j]+fdtd.deps[j] for j in range(3)]
            #print('np.max(eps_p_deps[j]:', [np.max(eps_p_deps[j]) for j in range(3)])
            #print('np.max(fdtd.deps[j]:', [np.max(fdtd.deps[j]) for j in range(3)])
            fdtd.Update_aA_bB(fdtd.E, fdtd.D, a=eps_p_deps)
    
        ## PEC
        fdtd.ResetSideWalls(fdtd.E)
        fdtd.ResetSideWalls(fdtd.D)
        t += dt/2
        
        ## Jm
        fdtd.UpdateSeperableFunc_Time(fdtd.Jm, Jm_0, fj_t, t)
        ## curl E
        curl_E = fdtd.GetCurlSide(fdtd.E)
        ## dF_M/dt + s_fm*F_M = -curl E - Jm
        fdtd.Update_aA_bB(fdtd.dF_H, curl_E, C_list=[fdtd.F_H, fdtd.Jm], b=-np.ones(3), c_list=[c_fh0, -np.ones(3)])
        ## dH/dt + s_h*H = dF_H/dt + s_fd*F_H
        fdtd.Update_adAdt_bB(fdtd.H, fdtd.dF_H, C_list=[fdtd.H, fdtd.F_H], c_list=[c_h, c_fh1])
        fdtd.Update_adAdt_bB(fdtd.F_H, fdtd.dF_H)
    
        if i%save_every==0:
            fdtd.SaveOutputs()
            n_saved += 1
    
toc = time.clock()
print('processing time: {}:{}'.format(int((toc-tic)/60), int((toc-tic)%60)))

fdtd.dr: [ 0.01        0.01007275]
self.E[0].shape: (1500, 79)
self.D[0].shape: (1500, 79)
self.Je[0].shape: (1500, 79)
self.eps[0].shape: (1500, 79)
self.H[0].shape: (1501, 78)
self.Jm[0].shape: (1501, 78)
inds_min_x: (125, 39) d: 0.005
inds_min_y: (125, 39) d: 0.00503637308537
inds_min_z: (125, 39) d: 0.0
je_0_max: [0.0, 0.0, 100.0]
inds_min_x: (125, 39) d: 0.00503637308537
inds_min_y: (125, 39) d: 0.005
inds_min_z: (125, 39) d: 0.00709683407267
jm_0_max: [0.0, 100.0, 0.0]
self.deps[0].shape: (1500, 79)
cutoffs [0.0, 2.8274333882308134, 5.6548667764616267, 8.4823001646924414]
del_eps: 0.2 omega_m: 1.298344160645776 k_m: -0.793524266037
self.dF_D[0].shape: (1500, 79)
self.F_D[0].shape: (1500, 79)
self.dF_H[0].shape: (1501, 78)
self.F_H[0].shape: (1501, 78)
r0: [ 0.  0.] r1: [ 15.          0.7856742]
r_T_1d: [ 14.125        0.58925565] r_R_1d: [ 0.875       0.19641855]
inds_min_x: (125, 39) d: 0.005
inds_min_y: (125, 39) d: 0.00503637308537
inds_min_z: (125, 39) d: 0.0
self.ViewPlanes:

### short script

In [3]:

tic = time.clock()
fdtd = FDTDYEE(r0, r1, dr, dt)

print('fdtd.dr:', fdtd.dr)

fdtd.AllocateSideArr_list(["E", "D", "Je", "eps"])
fdtd.AllocateFaceArr_list(["B", "H", "Jm"])
fdtd.SetSpatialPointsSide()
fdtd.SetSpatialPointsFace()

fdtd.SetupFdtdVars(FVTypes.E, None, var=fdtd.E)
fdtd.SetupFdtdVars(FVTypes.H, None, var=fdtd.H)
fdtd.SetupFdtdVars(FVTypes.D, None, var=fdtd.D)
fdtd.SetupFdtdVars(FVTypes.B, None, var=fdtd.B)

fdtd.save_every = save_every 

d_x_ = np.zeros(n_dim)
d_x_[0] = (r1[0]-r0[0])
d_x_pml = np.zeros(n_dim)
d_x_pml[0] = d_pml[0]

f_t = lambda t: np.sin(omega*t) 

## J
r_j = r_c - d_x_*0.45 + set_pml*d_x_pml
je_mag = 1.0/dr[0]
src_dir = 'z'
je_args = None
je_type = FVTypes.JePoint
if source_at_right:
    r_j = r_c + d_x_*0.45 - set_pml*d_x_pml
je_args = {'r0':r_j, 'mag':je_mag, 'f_t':f_t, 'src_dir':src_dir}
if set_sheet_src:
    assert n_dim>=2
    je_type = FVTypes.JeSheet
    je_mag = lambda r: je_mag*np.cos(r[1]-r_j[1])
    norm_dir = 'x'
    je_args = {'r0':r_j, 'mag':je_mag, 'f_t':f_t, 'src_dir':src_dir, 'norm_dir':norm_dir}
fdtd.SetupFdtdVars(je_type, je_args, var=fdtd.Je)


## M
r_m = r_c - d_x_*0.45 + set_pml*d_x_pml
jm_mag = -1.0/dr[0]
src_dir = 'y'
jm_args = None
jm_type = FVTypes.JmPoint
if source_at_right:
    r_m = r_c + d_x_*0.45 - set_pml*d_x_pml
    jm_mag *= -1.0
jm_args = {'r0':r_m, 'mag':jm_mag, 'f_t':f_t, 'src_dir':src_dir}
if set_sheet_src:
    assert n_dim>=2
    jm_type = FVTypes.JmSheet
    jm_mag = lambda r: jm_mag*np.cos(r[1]-r_j[1])
    norm_dir = 'x'
    jm_args = {'r0':r_m, 'mag':jm_mag, 'f_t':f_t, 'src_dir':src_dir, 'norm_dir':norm_dir}
fdtd.SetupFdtdVars(jm_type, jm_args, var=fdtd.Jm)


eps_args = {'r0':r0_eps, 'r1':r1_eps, 'mag_in':a_eps, 'mag_out':b_eps}
fdtd.SetupFdtdVars(FVTypes.EpsIsoBox, eps_args, var=fdtd.eps)

deps = None
f_deps = None
deps_out = []
if is_t_var:
    fdtd.AllocateSideArr_list(["deps"])
    del_eps = 0.2
    omega_m = 0.2*omega
    k_m = omega_m*np.sqrt(a_eps)
    if n_dim==2:
        k_ = ppwg_disp(omega, d=w_eps[1], eps_r=a_eps, n=1)
        k_m = -0.1*np.real(k_)
        dw = ppwg_dw(k=k_, dk=k_m, d=w_eps[1], eps_r=a_eps, n=[1,2])
        omega_m = np.real(dw)
        print('cutoffs', [ppwg_cutoff(r1[1], a_eps, i) for i in range(n_mode)])
    print('del_eps:', del_eps, 'omega_m:', omega_m, 'k_m:', k_m)
    a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r[0])
    if n_dim==1:
        a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r)
    deps_args = {'r0':r0_deps, 'r1':r1_deps, 'mag_in':a_deps, 'mag_out':0.0}
    fdtd.SetupFdtdVars(FVTypes.EpsIsoSTvarBox, deps_args, var=fdtd.deps)

    
if set_pml:
    pml_args = {'d_pml':d_pml, 's_pml':s_pml}
    fdtd.SetupFdtdVars(FVTypes.UPML, pml_args, var=None)

set_eps_out = True
set_j_out = False
set_T_1d = True
set_R_1d = True

r_R_1d = r_j - (r_j-d_pml)/2.0
r_T_1d = r1 - r_R_1d
if source_at_right:
    r_R_1d = r_j + (r1-d_pml-r_j)/2.0
    r_T_1d = r1 - r_R_1d
print('r0:', r0, 'r1:', r1)
print('r_T_1d:', r_T_1d, 'r_R_1d:', r_R_1d)

if n_dim==3:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j,  [{'A':fdtd.Je, 'A_dir':'z', 'O_dir':'x', 'name':'J'}])
elif n_dim<=2:
    fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':None, 'name':'E'}])
    if set_j_out:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.Je, 'A_dir':'z', 'O_dir':None, 'name':'J'}])
    if n_dim==1 and set_T_1d:
        fdtd.SetViewPlane_Side(r_T_1d, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'T'}])
    if n_dim==1 and set_R_1d:
        fdtd.SetViewPlane_Side(r_R_1d, [{'A':fdtd.E, 'A_dir':'z', 'O_dir':'x', 'name':'R'}])
        
if is_t_var and set_eps_out:
    if n_dim==3:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':'x', 'name':'Eps'}])
    elif n_dim<=2:
        fdtd.SetViewPlane_Side(r_j, [{'A':fdtd.deps, 'A_dir':'x', 'O_dir':None, 'name':'Eps'}])


        
fdtd.StepFields(n_t)
n_saved = fdtd.n_saved
    
toc = time.clock()
print('processing time: {}:{}'.format(int((toc-tic)/60), int((toc-tic)%60)))

fdtd.dr: [ 0.01        0.01007275]
self.E[0].shape: (1500, 79)
self.D[0].shape: (1500, 79)
self.Je[0].shape: (1500, 79)
self.eps[0].shape: (1500, 79)
self.B[0].shape: (1501, 78)
self.H[0].shape: (1501, 78)
self.Jm[0].shape: (1501, 78)
inds_min_x: (125, 39) d: 0.005
inds_min_y: (125, 39) d: 0.00503637308537
inds_min_z: (125, 39) d: 0.0
inds_min_x: (125, 39) d: 0.00503637308537
inds_min_y: (125, 39) d: 0.005
inds_min_z: (125, 39) d: 0.00709683407267
self.deps[0].shape: (1500, 79)
cutoffs [0.0, 2.8274333882308134, 5.6548667764616267, 8.4823001646924414]
del_eps: 0.2 omega_m: 1.298344160645776 k_m: -0.793524266037
r0: [ 0.  0.] r1: [ 15.          0.7856742]
r_T_1d: [ 14.125        0.58925565] r_R_1d: [ 0.875       0.19641855]
inds_min_x: (125, 39) d: 0.005
inds_min_y: (125, 39) d: 0.00503637308537
inds_min_z: (125, 39) d: 0.0
self.ViewPlanes:  dict_keys(['E'])
inds_min_x: (125, 39) d: 0.005
inds_min_y: (125, 39) d: 0.00503637308537
inds_min_z: (125, 39) d: 0.0
self.ViewPlanes:  dict_keys([

### super short script

In [2]:

fdtd = FDTDSimulator()

fdtd.SetSimulationBox(r0, r1, dr, dt)
fdtd.save_every = save_every 

d_x_ = np.zeros(n_dim)
d_x_[0] = (r1[0]-r0[0])
d_x_pml = np.zeros(n_dim)
d_x_pml[0] = d_pml[0]

f_t = lambda t: np.sin(omega*t) 

## J
r_j = r_c - d_x_*0.45 + set_pml*d_x_pml
je_mag = 1.0/dr[0]
src_dir = 'z'
je_args = None
je_type = FVTypes.JePoint
if source_at_right:
    r_j = r_c + d_x_*0.45 - set_pml*d_x_pml
je_args = {'r0':r_j, 'mag':je_mag, 'f_t':f_t, 'src_dir':src_dir}
if set_sheet_src:
    assert n_dim>=2
    je_type = FVTypes.JeSheet
    je_mag = lambda r: je_mag*np.cos(r[1]-r_j[1])
    norm_dir = 'x'
    je_args = {'r0':r_j, 'mag':je_mag, 'f_t':f_t, 'src_dir':src_dir, 'norm_dir':norm_dir}

fdtd.AddSources({'type':je_type, 'name':'Je', 'args':je_args})


## M
r_m = r_c - d_x_*0.45 + set_pml*d_x_pml
jm_mag = -1.0/dr[0]
src_dir = 'y'
jm_args = None
jm_type = FVTypes.JmPoint
if source_at_right:
    r_m = r_c + d_x_*0.45 - set_pml*d_x_pml
    jm_mag *= -1.0
jm_args = {'r0':r_m, 'mag':jm_mag, 'f_t':f_t, 'src_dir':src_dir}
if set_sheet_src:
    assert n_dim>=2
    jm_type = FVTypes.JmSheet
    jm_mag = lambda r: jm_mag*np.cos(r[1]-r_j[1])
    norm_dir = 'x'
    jm_args = {'r0':r_m, 'mag':jm_mag, 'f_t':f_t, 'src_dir':src_dir, 'norm_dir':norm_dir}
fdtd.AddSources({'type':jm_type, 'name':'Jm', 'args':jm_args})


eps_args = {'r0':r0_eps, 'r1':r1_eps, 'mag_in':a_eps, 'mag_out':b_eps}
fdtd.AddMaterial({'type':FVTypes.EpsIsoBox, 'name':'eps', 'args':eps_args})

deps = None
f_deps = None
deps_out = []
if is_t_var:
    del_eps = 0.2
    omega_m = 0.2*omega
    k_m = omega_m*np.sqrt(a_eps)
    if n_dim==2:
        k_ = ppwg_disp(omega, d=w_eps[1], eps_r=a_eps, n=1)
        k_m = -0.1*np.real(k_)
        dw = ppwg_dw(k=k_, dk=k_m, d=w_eps[1], eps_r=a_eps, n=[1,2])
        omega_m = np.real(dw)
        print('cutoffs', [ppwg_cutoff(r1[1], a_eps, i) for i in range(n_mode)])
    print('del_eps:', del_eps, 'omega_m:', omega_m, 'k_m:', k_m)
    a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r[0])
    if n_dim==1:
        a_deps = lambda r, t: del_eps*np.cos(omega_m*t - k_m*r)
    deps_args = {'r0':r0_deps, 'r1':r1_deps, 'mag_in':a_deps, 'mag_out':0.0}
    fdtd.AddMaterial({'type':FVTypes.EpsIsoSTvarBox, 'name':'deps', 'args':deps_args})

    
if set_pml:
    pml_args = {'d_pml':d_pml, 's_pml':s_pml}
    fdtd.AddPML({'type':FVTypes.UPML, 'name':'pml', 'args':pml_args})

set_eps_out = True
set_j_out = False
set_T_1d = True
set_R_1d = True

r_R_1d = r_j - (r_j-d_pml)/2.0
r_T_1d = r1 - r_R_1d
if source_at_right:
    r_R_1d = r_j + (r1-d_pml-r_j)/2.0
    r_T_1d = r1 - r_R_1d
print('r0:', r0, 'r1:', r1)
print('r_T_1d:', r_T_1d, 'r_R_1d:', r_R_1d)

if n_dim==3:
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j, 'args':{'A':'E', 'A_dir':'z', 'O_dir':'x', 'name':'E'}})
    if set_j_out:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j,  'args':{'A':'Je', 'A_dir':'z', 'O_dir':'x', 'name':'J'}})
elif n_dim<=2:
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j, 'args':{'A':'E', 'A_dir':'z', 'O_dir':None, 'name':'E'}})
    if set_j_out:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j, 'args':{'A':'Je', 'A_dir':'z', 'O_dir':None, 'name':'J'}})
    if n_dim==1 and set_T_1d:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_T_1d, 'args':{'A':'E', 'A_dir':'z', 'O_dir':'x', 'name':'T'}})
    if n_dim==1 and set_R_1d:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_R_1d, 'args':{'A':'E', 'A_dir':'z', 'O_dir':'x', 'name':'R'}})
        
if is_t_var and set_eps_out:
    if n_dim==3:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j, 'args':{'A':'deps', 'A_dir':'x', 'O_dir':'x', 'name':'Eps'}})
    elif n_dim<=2:
        fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_j, 'args':{'A':'deps', 'A_dir':'x', 'O_dir':None, 'name':'Eps'}})


        
fdtd.StepFields(n_t)
n_saved = fdtd.n_saved
    


del_eps: 0.2 omega_m: 1.2566370614359172 k_m: 1.77715317526
r0: [ 0.] r1: [ 6.36619772]
r_T_1d: [ 6.12746531] r_R_1d: [ 0.23873241]
self.E[0].shape: (4000,)
self.D[0].shape: (4000,)
self.B[0].shape: (4001,)
self.H[0].shape: (4001,)
self.Je[0].shape: (4000,)
inds_min_x: (249,) d: 0.000795774715459
inds_min_y: (250,) d: 1.66533453694e-16
inds_min_z: (250,) d: 1.66533453694e-16
self.Jm[0].shape: (4001,)
inds_min_x: (250,) d: 1.66533453694e-16
inds_min_y: (249,) d: 0.000795774715459
inds_min_z: (249,) d: 0.000795774715459
self.eps[0].shape: (4000,)
self.deps[0].shape: (4000,)
inds_min_x: (249,) d: 0.000795774715459
inds_min_y: (250,) d: 1.66533453694e-16
inds_min_z: (250,) d: 1.66533453694e-16
self.ViewPlanes:  dict_keys(['E'])
inds_min_x: (3850,) d: 0.000795774715459
inds_min_y: (3850,) d: 0.0
inds_min_z: (3850,) d: 0.0
self.ViewPlanes:  dict_keys(['E', 'T'])
inds_min_x: (149,) d: 0.000795774715459
inds_min_y: (150,) d: 8.32667268469e-17
inds_min_z: (150,) d: 8.32667268469e-17
self.ViewPl

### Plotting results

In [3]:
import matplotlib.pyplot as plt

r_E, E_out_list = fdtd.GetOutputs('E')
x_E, y_E, z_E = [None]*3
if n_dim==3:
    x_E, y_E, z_E = r_E
elif n_dim==2:
    x_E, y_E = r_E
elif n_dim==1:
    x_E = r_E[0]
e_max = [np.max(np.abs(e)) for e in E_out_list]
print('max(e_max):', max(e_max))
e_max = max(e_max)
    
x_J, y_J, z_J = [None]*3
j_max = None
J_out_list = None
if set_j_out:
    r_J, J_out_list = fdtd.GetOutputs('J')
    if n_dim==3:
        x_J, y_J, z_J = r_J
    elif n_dim==2:
        x_J, y_J = r_J
    elif n_dim==1:
        x_J = r_J[0]
    j_max = [np.max(np.abs(j)) for j in J_out_list]
    print('max(j_max):', max(j_max))
    j_max = max(j_max)



x_deps, y_deps, z_deps = [None]*3
deps_out_list = None
deps_max = None
if is_t_var and set_eps_out:
    r_deps, deps_out_list = fdtd.GetOutputs('Eps')
    if n_dim==3:
        x_deps, y_deps, z_deps = r_deps
    elif n_dim==2:
        x_deps, y_deps = r_deps
    elif n_dim==1:
        x_deps = r_deps[0]
    deps_max = [np.max(np.abs(de)) for de in deps_out_list]
    print('max(deps_max):', max(deps_max))
    deps_max = max(deps_max)


x_T = None
T_arr = None
if n_dim==1 and set_T_1d:
    r_T, T_out_list = fdtd.GetOutputs('T')
    x_T = r_T[0]
    T_arr = np.array([t[0] for t in T_out_list])
    T_max = np.max(np.abs(T_arr))
    print('max(T):', T_max)
    
x_R = None
R_arr = None
if n_dim==1 and set_R_1d:
    r_R, R_out_list = fdtd.GetOutputs('R')
    x_R = r_R[0]
    R_arr = np.array([t[0] for t in R_out_list])
    R_max = np.max(np.abs(R_arr))
    print('max(R):', R_max)
    
    
print(x_E.shape, E_out_list[0].shape)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams["figure.figsize"] = (5, 2)

if n_dim==2:
    fig = plt.imshow(E_out_list[-1].T, vmin=-e_max/10, vmax=e_max/10)
    ax = plt.gca()
    ax.set_adjustable('box-forced')
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    plt.show()

    if is_t_var and set_eps_out:
        #fig = plt.pcolor(x_deps, y_deps, deps_out_list[0])
        fig = plt.imshow(a_eps+deps_out_list[-1].T, vmin=a_eps-deps_max, vmax=a_eps+deps_max, aspect='auto')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        plt.show()

Saved outputs: dict_keys(['E', 'R', 'T', 'Eps'])
max(e_max): 1.3063977809
Saved outputs: dict_keys(['E', 'R', 'T', 'Eps'])
max(deps_max): 0.199999999998
Saved outputs: dict_keys(['E', 'R', 'T', 'Eps'])
max(T): 1.30517077246
Saved outputs: dict_keys(['E', 'R', 'T', 'Eps'])
max(R): 0.211792464198
(4001,) (4001,)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib

# Set up formatting for the movie files
#Writer = animation.writers['mencoder']    ## avconv ffmpeg mencoder
#writer = Writer(fps=15, metadata=dict(artist='Me'))
#writer = animation.FFMpegWriter()

plt_field = 'E'
save_anim = False
file_format = 'gif'   ## 'gif' / 'mp4'

logscale=False
log_0 = np.log(e_max)-10

use_subplots = False

font = {'family' : 'serif', 'weight' : 'normal', 'size'   : 14}
matplotlib.rc('font', **font)

fig = None
if n_dim>=2:
    def animate_E(i):
        plt.clf()
        #fig = plt.pcolor(y_E, z_E, E_out_list[i])
        if not logscale:
            #fig = plt.imshow(np.abs(E_out_list[i]).T, vmin=0.0, vmax=e_max/100)
            fig = plt.imshow(E_out_list[i].T, origin='lower', vmin=-e_max/10, vmax=e_max/10, aspect='auto')
        else:
            E_log = np.log(np.abs(E_out_list[i]))
            E_log = (E_log>=log_0)*E_log + (E_log<log_0)*(log_0)
            fig = plt.imshow(E_log.T, origin='lower', vmin=log_0, vmax=np.log(e_max), aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        return fig

    def animate_Eps(i):
        plt.clf()
        fig = plt.imshow(deps_out_list[i].T, origin='lower', vmin=-deps_max, vmax=deps_max, aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        return fig

    def animate_J(i):
        plt.clf()
        #fig = plt.pcolor(y_J, z_J, J_out_list[i])
        fig = plt.imshow(J_out_list[i])
        axes = plt.gca()
        CB = plt.colorbar(fig, shrink=0.8, extend='both')
        return fig


    fig = plt.figure()
    axes = plt.gca()
    anim = None
    if plt_field=='E':
        anim = animation.FuncAnimation(fig, animate_E, frames=n_saved, interval=1, repeat=False)
    elif plt_field=='Eps':
        anim = animation.FuncAnimation(fig, animate_Eps, frames=n_saved, interval=1)
    elif plt_field=='J':
        anim = animation.FuncAnimation(fig, animate_J, frames=n_saved, interval=1)
    else:
        raise ValuError()

    if save_anim:
        if file_format=='gif':
            anim.save('other/e-2d.gif', writer="imagemagick", fps=15, dpi=200)
        elif file_format=='mp4':
            anim.save('other/e-2d.mp4', writer="mencoder", fps=15, dpi=200)
    plt.show()
elif n_dim==1:
    
    if set_T_1d:
        f__0 = omega/(2.0*np.pi)
        d__f = omega_m/((2.0*np.pi))
        T_arr = np.trim_zeros(T_arr)
        plt.plot(T_arr)
        plt.show()
        T_fft = np.fft.fft(T_arr)
        T_fft = np.abs(T_fft)/len(T_fft)
        freq = np.fft.fftfreq(T_fft.shape[-1], dt)
        plt.plot(freq, 2.0*T_fft)
        plt.gca().set_xlim([f__0-3.0*d__f, f__0+3.0*d__f])
        plt.gca().set_ylim([0, T_max])
        plt.show()
        plt.plot(freq, np.log10(2.0*T_fft))
        plt.gca().set_xlim([f__0-3.0*d__f, f__0+3.0*d__f])
        plt.gca().set_ylim([-2.0, 0.0])
        plt.show()

    fig, axarr = None, None
    def animate_E(i):
        #global fig
        if not is_t_var:
            plt.clf()
            fig = plt.plot(x_E, E_out_list[i])
            axes = plt.gca()
            axes.set_ylim([-e_max, e_max])
            return fig
        else:
            if not use_subplots:
                plt.clf()
                fig = None
                if plt_field=='E':
                    fig = plt.plot(x_E, E_out_list[i])
                    ax = plt.gca()
                    ax.set_ylim([-e_max, e_max])
                    ax.axes.get_xaxis().set_ticks([])
                    ax.axes.get_yaxis().set_ticks([])
                elif plt_field=='Eps':
                    fig = plt.plot(x_deps, deps_out_list[i], 'r')
                    ax = plt.gca()
                    ax.set_ylim([-deps_max, deps_max])
                    ax.axes.get_xaxis().set_ticks([])
                #fig.set_tight_layout(True)
                ax = plt.gca()
                ax.set_adjustable('box-forced')
                plt.axvline(x=r0_deps[0], linewidth=1.0, linestyle='-.',color='r')
                plt.axvline(x=r1_deps[0], linewidth=1.0, linestyle='-.',color='r')
                return fig
            else:
                plt.clf()
                fig, axarr = plt.subplots(2, sharex=True)
                axarr[0].plot(x_E, E_out_list[i])
                axarr[0].set_ylim([-e_max, e_max])
                axarr[1].plot(x_deps, deps_out_list[i], 'r')
                axarr[1].set_ylim([-deps_max, deps_max])
                return fig
            

    def animate_J(i):
        plt.clf()
        fig = plt.plot(x_J, J_out_list[i])
        axes = plt.gca()
        axes.set_ylim([-j_max, j_max])
        return fig

    anim = None
    if is_t_var and use_subplots:
        fig, axarr = plt.subplots(2, sharex=True)
    else:
        fig = plt.figure()
    
    if plt_field=='E' or plt_field=='Eps':
        anim = animation.FuncAnimation(fig, animate_E, frames=n_saved, interval=1)
    elif plt_field=='J':
        anim = animation.FuncAnimation(fig, animate_J, frames=n_saved, interval=1)
    else:
        raise ValuError()

    if save_anim:
        if file_format=='gif':
            anim.save('other/e-1d.gif', writer="imagemagick", fps=15, dpi=200)
        elif file_format=='mp4':
            anim.save('other/e-1d.mp4', writer="mencoder", fps=15, dpi=200)
    plt.show()
    
    

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def ppwg_disp(omega, d, eps_r, n):
    k_c = n*np.pi/d
    k_sq = omega**2*eps_r - k_c**2 + 0j
    k = (np.real(k_sq)>=0)*np.sqrt(k_sq)
    return k


def ppwg_disp_kin(k, d, eps_r, n):
    k_c = n*np.pi/d
    w_sq = (k**2 + k_c**2 + 0j)/eps_r
    w = (np.real(w_sq)>=0)*np.sqrt(w_sq)
    return w

def ppwg_cutoff(d, eps_r, n):
    k_c = n*np.pi/d
    w_c = k_c/np.sqrt(eps_r)
    return w_c

def ppwg_cutoff_d(w_c, eps_r, n):
    k_c = w_c*np.sqrt(eps_r)
    d = n*np.pi/k_c
    return d

def ppwg_dw(k, dk, d, eps_r, n):
    n_0, n_1 = n
    w_0 = ppwg_disp_kin(k, d, eps_r, n_0)
    w_1 = ppwg_disp_kin(k+dk, d, eps_r, n_1)
    dw = np.real(w_1 - w_0)
    return dw

    
n_pt = 1000
d = 1.0
eps_r = 2.0
k = np.linspace(-10.0, 10.0, n_pt)

n_mode = 4
omega = [np.real(ppwg_disp_kin(k, d, eps_r, i)) for i in range(n_mode)] 

print([ppwg_cutoff(d, eps_r, i) for i in range(n_mode)])

for i in range(n_mode):
    plt.plot(k, omega[i])
    #plt.plot(-k[i], omega, 'b')

#plt.gca().get_xaxis().set_visible(False)
#plt.gca().get_yaxis().set_visible(False)
plt.axvline(x=0.0, linewidth=0.5, color='k')

print(ppwg_dw(k=1, dk=-0.5, d=d, eps_r=eps_r, n=[1, 2]))


w_1 = 5.0
k_1 = ppwg_disp(w_1, d, eps_r, n=1).real
print('w={}  k={}'.format(w_1, k_1))
dk = -0.5
dw = ppwg_dw(k_1, dk, d, eps_r, n=[1, 2])
print('dw={}  dk={}'.format(dw, dk))

n_jump=3
ksp = [k_1+i*dk for i in range(n_jump)]
ksm = [-k_1+i*dk for i in range(n_jump)]
ws = [w_1+i*dw for i in range(n_jump)]

plt.plot(ksp, ws, 'or')
plt.plot(ksm, ws, 'or')
plt.grid()

plt.show()


[0.0, 2.2214414690791831, 4.4428829381583661, 6.6643244072375483]
2.1256619574756757
w=5.0  k=6.334855609949657
dw=1.0631657404906232  dk=-0.5
